In [ ]:
!pip install -q tensorflow scikit-learn scipy h5py

import tensorflow as tf
import numpy as np
import h5py
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, Model
import os

print(f'TensorFlow: {tf.__version__}')
print(f'GPU: {tf.config.list_physical_devices("GPU")}')

os.makedirs('models', exist_ok=True)

In [ ]:
# ENHANCED DATA GENERATION (60K samples)
print('\nGenerating 60,000 samples...')

CLASSES = ['blast', 'gunshot', 'artillery', 'vehicle_crash', 'fall', 'normal']

def generate_sample(impact_type):
    params = {
        'blast': {'amp': (150,300), 'freq': (50,150), 'decay': 15, 'sev': (0.6,1.0)},
        'gunshot': {'amp': (40,100), 'freq': (100,300), 'decay': 25, 'sev': (0.5,0.9)},
        'artillery': {'amp': (200,400), 'freq': (30,100), 'decay': 12, 'sev': (0.6,1.0)},
        'vehicle_crash': {'amp': (20,80), 'freq': (5,30), 'decay': 5, 'sev': (0.3,0.7)},
        'fall': {'amp': (15,60), 'freq': (3,20), 'decay': 8, 'sev': (0.3,0.7)},
        'normal': {'amp': (0.5,3), 'freq': (0.1,5), 'decay': 2, 'sev': (0.0,0.2)}
    }
    
    p = params[impact_type]
    t = np.linspace(0, 1, 200)
    
    # Multi-frequency components
    amp = np.random.uniform(*p['amp'])
    f1 = np.random.uniform(*p['freq'])
    f2 = f1 * np.random.uniform(1.3, 1.8)
    f3 = f1 * np.random.uniform(0.5, 0.8)
    
    start = np.random.randint(10, 50)
    dur = np.random.randint(30, 120)
    
    sig = np.zeros(200)
    impact_t = t[start:start+dur] - t[start]
    
    # Multi-harmonic signal
    sig[start:start+dur] = (
        amp * np.sin(2*np.pi*f1*impact_t) * np.exp(-p['decay']*impact_t) +
        amp * 0.3 * np.sin(2*np.pi*f2*impact_t) * np.exp(-p['decay']*impact_t*1.2) +
        amp * 0.2 * np.sin(2*np.pi*f3*impact_t) * np.exp(-p['decay']*impact_t*0.8)
    )
    
    sig += np.random.normal(0, amp*0.12, 200)
    
    # 3-axis IMU
    accel_z = sig
    accel_x = np.roll(sig, np.random.randint(3,8)) * np.random.uniform(0.2,0.6) + np.random.normal(0, 0.5, 200)
    accel_y = np.roll(sig, np.random.randint(3,8)) * np.random.uniform(0.2,0.6) + np.random.normal(0, 0.5, 200)
    
    gyro_x = np.gradient(accel_y) * np.random.uniform(10,15) + np.random.normal(0, 2, 200)
    gyro_y = np.gradient(accel_x) * np.random.uniform(10,15) + np.random.normal(0, 2, 200)
    gyro_z = np.gradient(accel_z) * np.random.uniform(10,15) + np.random.normal(0, 2, 200)
    
    mag = np.tile([30, 20, -40], (200, 1)) + np.random.normal(0, 3, (200, 3))
    
    # Vitals
    sev = np.random.uniform(*p['sev'])
    
    hr_base = np.random.uniform(60, 80)
    hr = hr_base + sev*85*(1-np.exp(-np.arange(200)/100)) + 3*np.sin(2*np.pi*0.25*t) + np.random.normal(0, 2, 200)
    hr = np.clip(hr, 40, 180)
    
    spo2 = 97 - sev*18*(1-np.exp(-np.arange(200)/180)) + np.random.normal(0, 0.5, 200)
    spo2 = np.clip(spo2, 70, 100)
    
    br = 14 + sev*14*(1-np.exp(-np.arange(200)/120)) + np.random.normal(0, 1, 200)
    br = np.clip(br, 8, 40)
    
    temp = 36.6 + sev*0.6*(1-np.exp(-np.arange(200)/250)) + np.random.normal(0, 0.08, 200)
    temp = np.clip(temp, 34, 39)
    
    sample = np.column_stack([accel_x, accel_y, accel_z, gyro_x, gyro_y, gyro_z, mag, hr, spo2, br, temp])
    return sample.astype(np.float32), sev

# Generate dataset
X_all = []
y_type_all = []
y_sev_all = []

for i, cls in enumerate(CLASSES):
    print(f'[{i+1}/6] {cls}... ', end='')
    for j in range(10000):
        if (j+1) % 2000 == 0:
            print(f'{j+1}... ', end='')
        sample, sev = generate_sample(cls)
        X_all.append(sample)
        
        label = np.zeros(6)
        label[i] = 1
        y_type_all.append(label)
        y_sev_all.append(sev)
    print('Done')

X = np.array(X_all, dtype=np.float32)
y_type = np.array(y_type_all, dtype=np.float32)
y_sev = np.array(y_sev_all, dtype=np.float32)

print(f'\nDataset: {X.shape}')
print('Normalizing...')

# Normalize
X_mean = X.mean(axis=(0,1), keepdims=True)
X_std = X.std(axis=(0,1), keepdims=True) + 1e-8
X = (X - X_mean) / X_std

np.savez('models/norm.npz', mean=X_mean, std=X_std)
print('✓ Data ready')

In [ ]:
# SPLIT DATA
X_train, X_temp, y_type_train, y_type_temp, y_sev_train, y_sev_temp = train_test_split(
    X, y_type, y_sev, test_size=0.3, random_state=42, stratify=y_type.argmax(1)
)

X_val, X_test, y_type_val, y_type_test, y_sev_val, y_sev_test = train_test_split(
    X_temp, y_type_temp, y_sev_temp, test_size=0.5, random_state=42, stratify=y_type_temp.argmax(1)
)

print(f'Train: {len(X_train)}, Val: {len(X_val)}, Test: {len(X_test)}')

In [ ]:
# SIMPLE DATA AUGMENTATION
def augment_data(X, y_type, y_sev):
    """Simple augmentation - tested and works"""
    X_aug = []
    y_type_aug = []
    y_sev_aug = []
    
    for i in range(len(X)):
        # Original
        X_aug.append(X[i])
        y_type_aug.append(y_type[i])
        y_sev_aug.append(y_sev[i])
        
        # Augmented (50% of samples)
        if i % 2 == 0:
            x_new = X[i].copy()
            
            # Time shift
            shift = np.random.randint(-8, 8)
            x_new = np.roll(x_new, shift, axis=0)
            
            # Scale
            scale = np.random.uniform(0.92, 1.08)
            x_new *= scale
            
            # Noise
            x_new += np.random.normal(0, 0.02, x_new.shape)
            
            X_aug.append(x_new)
            y_type_aug.append(y_type[i])
            y_sev_aug.append(y_sev[i])
    
    return np.array(X_aug), np.array(y_type_aug), np.array(y_sev_aug)

print('Augmenting training data...')
X_train_aug, y_type_train_aug, y_sev_train_aug = augment_data(X_train, y_type_train, y_sev_train)
print(f'Augmented: {len(X_train)} → {len(X_train_aug)} samples')

In [ ]:
# ADVANCED MODEL
def residual_block(x, filters, dilation=1):
    shortcut = x
    
    x = layers.Conv1D(filters, 3, padding='same', dilation_rate=dilation)(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    
    x = layers.Conv1D(filters, 3, padding='same', dilation_rate=dilation)(x)
    x = layers.BatchNormalization()(x)
    
    if shortcut.shape[-1] != filters:
        shortcut = layers.Conv1D(filters, 1)(shortcut)
    
    x = layers.Add()([shortcut, x])
    return layers.ReLU()(x)

print('\nBuilding model...')

inputs = layers.Input(shape=(200, 13))

# Initial conv
x = layers.Conv1D(64, 7, padding='same')(inputs)
x = layers.BatchNormalization()(x)
x = layers.ReLU()(x)

# Stage 1
x = residual_block(x, 64, dilation=1)
x = residual_block(x, 64, dilation=2)
x = layers.MaxPooling1D(2)(x)

# Stage 2
x = residual_block(x, 128, dilation=1)
x = residual_block(x, 128, dilation=2)
x = residual_block(x, 128, dilation=4)
x = layers.MaxPooling1D(2)(x)

# Stage 3
x = residual_block(x, 256, dilation=1)
x = residual_block(x, 256, dilation=2)
x = residual_block(x, 256, dilation=4)

# Stage 4
x = residual_block(x, 512, dilation=1)
x = residual_block(x, 512, dilation=2)

# Attention
attn = layers.MultiHeadAttention(num_heads=16, key_dim=32)(x, x)
x = layers.Add()([x, attn])
x = layers.LayerNormalization()(x)

# SE block
se = layers.GlobalAveragePooling1D()(x)
se = layers.Dense(32, activation='relu')(se)
se = layers.Dense(512, activation='sigmoid')(se)
se = layers.Reshape((1, 512))(se)
x = layers.Multiply()([x, se])

# Pooling
avg_pool = layers.GlobalAveragePooling1D()(x)
max_pool = layers.GlobalMaxPooling1D()(x)
x = layers.concatenate([avg_pool, max_pool])

# Classification head
x = layers.Dense(768, activation='relu')(x)
x = layers.BatchNormalization()(x)
x = layers.Dropout(0.5)(x)

x = layers.Dense(512, activation='relu')(x)
x = layers.BatchNormalization()(x)
x = layers.Dropout(0.4)(x)

x = layers.Dense(256, activation='relu')(x)
x = layers.BatchNormalization()(x)
x = layers.Dropout(0.3)(x)

# Outputs
impact_type = layers.Dense(6, activation='softmax', name='impact_type')(x)
severity = layers.Dense(1, activation='sigmoid', name='severity')(x)

model = Model(inputs=inputs, outputs=[impact_type, severity])

# Compile
model.compile(
    optimizer=tf.keras.optimizers.Adam(0.001, clipnorm=1.0),
    loss={
        'impact_type': tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.1),
        'severity': 'mse'
    },
    loss_weights={'impact_type': 1.0, 'severity': 0.2},
    metrics={'impact_type': ['accuracy'], 'severity': ['mae']}
)

params = sum([np.prod(v.shape) for v in model.trainable_weights])
print(f'Parameters: {params:,}')
print('✓ Model ready')

In [ ]:
# TRAIN
print('\n' + '='*70)
print('TRAINING')
print('='*70)

callbacks = [
    tf.keras.callbacks.EarlyStopping(
        monitor='val_impact_type_accuracy',
        patience=30,
        restore_best_weights=True,
        mode='max',
        verbose=1
    ),
    tf.keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=12,
        min_lr=1e-7,
        verbose=1
    ),
    tf.keras.callbacks.ModelCheckpoint(
        'models/best_model.h5',
        monitor='val_impact_type_accuracy',
        save_best_only=True,
        mode='max',
        verbose=1
    )
]

history = model.fit(
    X_train_aug,
    {'impact_type': y_type_train_aug, 'severity': y_sev_train_aug},
    validation_data=(X_val, {'impact_type': y_type_val, 'severity': y_sev_val}),
    epochs=200,
    batch_size=32,
    callbacks=callbacks,
    verbose=1
)

print('\n✓ Training complete')

In [ ]:
# EVALUATE
print('\n' + '='*70)
print('FINAL EVALUATION')
print('='*70)

results = model.evaluate(
    X_test,
    {'impact_type': y_type_test, 'severity': y_sev_test},
    batch_size=32,
    verbose=0
)

test_acc = results[3]  # impact_type_accuracy

print(f'\nTEST ACCURACY: {test_acc:.4f} ({test_acc*100:.2f}%)')
print(f'TARGET: 0.9500 (95.00%)')

if test_acc >= 0.95:
    print('✓ TARGET ACHIEVED!')
else:
    print(f'✗ Below target by {(0.95-test_acc)*100:.2f}%')

print('='*70)

In [ ]:
# SAVE MODEL
print('\nSaving model...')

model.save('models/final_model.h5')
print('✓ Saved: models/final_model.h5')

# Try TFLite conversion (might fail, but we have .h5 backup)
try:
    print('\nAttempting TFLite conversion...')
    converter = tf.lite.TFLiteConverter.from_keras_model(model)
    converter.optimizations = [tf.lite.Optimize.DEFAULT]
    tflite_model = converter.convert()
    
    with open('models/impact_classifier.tflite', 'wb') as f:
        f.write(tflite_model)
    
    print(f'✓ TFLite saved ({len(tflite_model)/1024:.1f} KB)')
except Exception as e:
    print(f'⚠ TFLite conversion failed (expected): {str(e)[:100]}')
    print('  Use the .h5 model instead')

In [ ]:
# DOWNLOAD
from google.colab import files

print('\nDownloading files...')

# Download whichever model exists
if os.path.exists('models/impact_classifier.tflite'):
    files.download('models/impact_classifier.tflite')
    print('✓ Downloaded: impact_classifier.tflite')
else:
    files.download('models/final_model.h5')
    print('✓ Downloaded: final_model.h5')

files.download('models/norm.npz')
print('✓ Downloaded: norm.npz (normalization parameters)')

print(f'\n{"="*70}')
print(f'FINAL ACCURACY: {test_acc*100:.2f}%')
print('='*70)